# Быстро строим деревья


Деревья глубины 1 обычно называют решающими пнями. 
Как Вы думаете, какая сложность построения решающего пня?


Представим, что мы решаем задачу регрессии, считаем для одного сплита. 

$$Q = F(R_m) - \frac{N_l}{N_m} F(R_l) - \frac{N_r}{N_m} F(R_r) $$

$$F(R) = \frac{1}{N} \sum_{i=1}^{N}(mean(y) - y_i) ^ 2 $$

$$Q = F(R_m) - \frac{1}{N_m} [ \sum_{i \in R_l}(mean(y_l) - y_i) ^ 2  +  \sum_{i \in R_r}(mean(y_r) - y_i) ^ 2 ]  $$

$F(R_m) и N_m$ от сплита не зависит, можно отбросить. Осталось научиться быстро считать такие выражения:
$\sum_{i \in R_l}(mean(y_l) - y_i) ^ 2$ для всех порогов. 

Если делать в лоб, то получится $O(||R||^2)$, нам не нравится, очень долго. Каждый раз пересчитывать довольно глупо, потому что мы точно знаем, какой именно объект при изменении сплита перешел из левой части в правую!




Воспользуемся формулой для дисперсии:
$$D(R) = mean(y ^ 2) -  mean(y) ^ 2 =   \frac{1}{N} \sum_{i \in R} y_i^2 -  \frac{1}{N ^ 2} (\sum_{i \in R} y_i) ^2$$

Но у нас не дисперсия, а дисперсия без деления на число объектов, то есть:
$$\sum_{i \in R_l}(mean(y_l) - y_i) ^ 2 =   \sum_{i \in R_k} y_i^2 -  \frac{1}{N_l} (\sum_{i \in R_l} y_i) ^2 $$
Аналогично для правого сплита. 

Вот мы и получили более простой алгоритм подсчета сплита:

1) Отсортировали фичу и таргет по фиче

2) Сначала считаем, что порог это минимальное значение, то есть левый кусок пустой, правый все значения. Храним для каждой из частей
а) Сумму таргетов
б) Сумму квадратов
в) Число объектов

3) Взяли следующее значение в отсортированном списке, это значение ушло от правого куска в левое. Обновили а) б) в) в обоих кусках

4) Посчитали  для каждого куска б) - а) ** 2 / в), сложили для обоих кусков. Если эта сумма маеньше лучшего значения, то это лучший сплит из текущих


Реализуйте алгоритм, сравните качество со sklearn.

Если получилось одинаково, Вы молодец!!!

In [72]:
import numpy as np


In [132]:


def get_threshold(feature, y, train_mode=None):
    best_thr = None
    ind = np.argsort(feature)
    feature = feature[ind]
    y = y[ind]
    N_l, N_r, y_r, y_l, y_2_r, y_2_l = 0, y.shape[0], y.sum(),0, np.square(y).sum(),0
    mean_r =(y_2_r - (y_r ** 2) / N_r)
    mean_l = 0
    min_ = mean_r + mean_l
    best_thr = (min_, y[0], mean_l, mean_r)
    i = 0
    for j in y:  
        N_r -=1
        N_l += 1
        y_r -= j
        y_l += j
        y_2_r -= j**2
        y_2_l += j**2
        cur_mean_l = (y_2_l - (y_l ** 2) / N_l)
        if N_r:
            cur_mean_r = (y_2_r - (y_r ** 2) / N_r)
        else:
            cur_mean_r = 0
        cur_min = cur_mean_l + cur_mean_r 
        if cur_min < min_:
            min_ = cur_min
            
            best_thr = (min_, feature[i], y_l / N_l, y_r / N_r)  
        i += 1
    return best_thr


def train_stump(X, y):
    best_f = 0
    best_thr = 0
    best_error = float('inf')
    left_mean, right_mean = 0, 0
    for j in range(X.shape[1]):
        f_score, f_thr, new_l_m,new_r_m  = get_threshold(X[:, j], y)
        if f_score < best_error:
            best_error = f_score
            best_f = j
            best_thr = f_thr
            left_mean = new_l_m
            right_mean = new_r_m
    return best_f, best_thr, left_mean, right_mean


    

In [133]:
from sklearn.datasets import load_boston
X, y = load_boston(return_X_y=True)


In [134]:
def my_predict(X, thr):
    res = np.arange(X.shape[0])
    for i in range(X.shape[0]):
        if X[i][thr[0]] > thr[1]:
            res[i] = thr[3]
        else:
            res[i] = thr[2]
    return res
    

In [135]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
regressor = DecisionTreeRegressor(random_state=0).fit(X, y)
print(mean_squared_error(y, regressor.predict(X)))
regressor_stump = DecisionTreeRegressor(random_state=0, max_depth=1).fit(X, y)
print(mean_squared_error(y, regressor_stump.predict(X)))
thr = train_stump(X, y)
#print(thr)
print(mean_squared_error(y, my_predict(X, thr)))


0.0
46.19909167710848
(5, 6.939, 19.933720930232564, 37.23815789473695)
46.948498023715416
